In [1]:
from mdagent import MDAgent
import os 

In [2]:
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
from robustness_prompts import get_prompt

prompt_3_descriptive = get_prompt("descriptive", 3)
prompt_3_natural = get_prompt("natural", 3)

prompt_3_descriptive, prompt_3_natural

('Complete all of the following tasks: 1. Download the PDB file 1LYZ. 2. Report the secondary structure assignments of the downloaded PDB structure. 3. Simulate for 1ps at 300 K.',
 'Simulate 1LYZ for 1ps at 300 K. Report the secondary structure assignments of the downloaded PDB structure.')

In [3]:
llm_model = "accounts/fireworks/models/llama-v3p1-405b-instruct"
tools = "all"

In [4]:
agent_1 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools, use_memory=False)
agent_1.run(prompt_3_descriptive)

Thought: First, I need to download the PDB file 1LYZ. Then, I will report the secondary structure assignments of the downloaded PDB structure. Finally, I will simulate for 1ps at 300 K.

Action:
```
{
  "action": "PDBFileDownloader",
  "action_input": "1LYZ"
}
```
PDB file found with this ID: 1LYZ
Thought: Now that I have downloaded the PDB file 1LYZ, I need to report the secondary structure assignments of the downloaded PDB structure.

Action:
```
{
  "action": "ComputeDSSP",
  "action_input": {
    "traj_file": "1LYZ_101413",
    "top_file": "1LYZ_101413",
    "target_frames": "first"
  }
}
```Thought: Now that I have reported the secondary structure assignments of the downloaded PDB structure, I need to simulate for 1ps at 300 K.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1LYZ_101413",
    "forcefield_files": ["amber99sbildn.xml"],
    "save": true,
    "system_params": {
      "nonbondedMethod": "NoCutoff",
      "constraints": "None",
    

System built successfully
<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7f676531ed00> >
Setting up integrator...
Creating simulation...
simulation set!
Performing energy minimization...
Minimization complete!
Initial Positions saved to initial_positions.pdb
Equilibrating...
Simulating...
Done!
Standalone simulation script written to /gpfs/fs2/scratch/qcampbe2/repos/md-agent/ckpt/ckpt_114/simulations/NVT_1LYZ_101426_20250124_101430.py
['prompt7.ipynb', 'prompt1.ipynb', 'prompt3.ipynb', '.ipynb_checkpoints', 'LOG_sim0_101430_1LYZ_101426_20250124_101432.txt', 'prompt10.ipynb', 'prompt9.ipynb', 'TRAJ_sim0_101430_1LYZ_101426_20250124_101432.dcd', 'prompt2.ipynb', 'prompt8.ipynb', 'TOP_sim0_101430_1LYZ_101426_20250124_101432.pdb', 'prompt6.ipynb', 'prompt4.ipynb', 'prompt5.ipynb']
Thought: The simulation was successful, and I have obtained the simulation trajectory, state log, and pdb frames for protein 1LYZ_101426. I also have a standalone script written with 

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [5]:
registry_1 = agent_1.path_registry
print(registry_1.list_path_names_and_descriptions().replace(",", "\n"))

Files found in registry: 1LYZ_101413: PDB file downloaded from RSCB
 PDBFile ID: 1LYZ_101413
 rec0_101416: dssp values for trajectory with id: 1LYZ_101413
 1LYZ_101426: Cleaned File:  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_101430: Initial positions for simulation sim0_101430
 sim0_101430: Basic Simulation of Protein 1LYZ_101426
 rec0_101432: Simulation trajectory for protein 1LYZ_101426 and simulation sim0_101430
 rec1_101432: Simulation state log for protein 1LYZ_101426 and simulation sim0_101430
 rec2_101432: Simulation pdb frames for protein 1LYZ_101426 and simulation sim0_101430


In [16]:
# make sure pdb was downloaded
assert os.path.exists(registry_1.get_mapped_path("1LYZ_101413"))

In [17]:
# make sure dssp was computed correctly
from mdagent.tools.base_tools import ComputeDSSP

dssp = ComputeDSSP(registry_1)
dssp._run(traj_file= "1LYZ_101413", target_frames="first")

"{'residues in helix': 50, 'residues in strand': 14, 'residues in coil': 65, 'residues not assigned, not a protein residue': 101}"

In [18]:
# make sure trajectory and topology exist
traj_path_1 = registry_1.get_mapped_path("rec0_101432")
top_path_1 = registry_1.get_mapped_path("top_sim0_101430")

assert os.path.exists(traj_path_1)
assert os.path.exists(top_path_1)

In [9]:
agent_2 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools, use_memory=False)
agent_2.run(prompt_3_natural)

Thought: First, I need to download the PDB file for 1LYZ. Then, I can use the downloaded file to run a simulation for 1ps at 300 K. After the simulation, I will compute the secondary structure assignments of the PDB structure.

Action:
```
{
  "action": "PDBFileDownloader",
  "action_input": "1LYZ"
}
```
PDB file found with this ID: 1LYZ
Now that I have the PDB file, I can set up and run a simulation for 1ps at 300 K.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1LYZ_101446",
    "forcefield_files": ["amber99sbildn.xml"],
    "save": true,
    "system_params": {
      "nonbondedMethod": "NoCutoff",
      "constraints": "None",
      "rigidWater": false
    },
    "integrator_params": {
      "integrator_type": "LangevinMiddle",
      "Temperature": "300 * kelvin",
      "Friction": "1.0 / picoseconds",
      "Timestep": "0.002 * picoseconds",
      "Pressure": "1.0 * bar"
    },
    "simulation_params": {
      "Ensemble": "NVT",
      "Number of

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [10]:
registry_2 = agent_2.path_registry
print(registry_2.list_path_names_and_descriptions().replace(",", "\n"))

Files found in registry: 1LYZ_101446: PDB file downloaded from RSCB
 PDBFile ID: 1LYZ_101446
 1LYZ_101456: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_101500: Initial positions for simulation sim0_101500
 sim0_101500: Basic Simulation of Protein 1LYZ_101456
 rec0_101500: Simulation trajectory for protein 1LYZ_101456 and simulation sim0_101500
 rec1_101500: Simulation state log for protein 1LYZ_101456 and simulation sim0_101500
 rec2_101500: Simulation pdb frames for protein 1LYZ_101456 and simulation sim0_101500
 rec0_103954: dssp values for trajectory with id: rec2_101500


In [14]:
# make sure pdb was downloaded
assert os.path.exists(registry_2.get_mapped_path("1LYZ_101446"))

In [15]:
# make sure dssp was computed correctly
from mdagent.tools.base_tools import ComputeDSSP

dssp = ComputeDSSP(registry_2)
dssp._run(traj_file= "1LYZ_101446", target_frames="first")

"{'residues in helix': 50, 'residues in strand': 14, 'residues in coil': 65, 'residues not assigned, not a protein residue': 101}"

In [19]:
# make sure trajectory and topology exist
traj_path_2 = registry_2.get_mapped_path("rec0_101500")
top_path_2 = registry_2.get_mapped_path("top_sim0_101500")

assert os.path.exists(traj_path_2)
assert os.path.exists(top_path_2)